In [30]:
import numpy as np
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup

import requests

from time import sleep

from datetime import datetime
import pickle
import os
import re

from hab.airtable.airtable import AirtableHAB
from hab.serpapi import serpapi, locations
from hab.clean_functions import clean

In [2]:
today = datetime.now().strftime("%Y%m%d")
# print(today)

with open(file = "serpapi_key.txt", mode = "r") as file:
    api_key = file.read()
    
# print(api_key)

lista_puestos = pd.read_excel("Lista_Puestos.xlsx", header = None)

lista_puestos.columns = ["jobs"]

In [3]:
hab1 = AirtableHAB(table_name = "LATAM 001")
hab2 = AirtableHAB(table_name = "LATAM 002")

In [4]:
%%time

df1 = hab1.airtable_get()
# df2 = hab2.airtable_get()


# df = pd.concat([df1, df2], axis = 0, ignore_index = True)

# df.head()

CPU times: total: 23.1 s
Wall time: 6min 7s


In [5]:
df1["years_of"] = df1["description"].str.contains("años de|years of|years experience|años experiencia")

In [6]:
df1["years_of"].sum()

18081

In [7]:
def find_years_of_experience(string: str):
    
    list_strings = ["años de", "years of", "years experience", "años experiencia"]
    
    string = string.lower()
    
    years = [string[string.find(s) - 5 : string.find(s) + len(s) + 1] for s in list_strings if string.find(s) != -1]
    
    numeros = [re.findall(r"\d+", y) for y in years]

    numeros = [[int(n) for n in num if 0 < int(n) < 13] for num in numeros]
    
    numeros = [max(num) if num else np.nan for num in numeros]

    return max(numeros) if numeros else np.nan

In [8]:
def experience_level(num):
    
    if not pd.isna(num):
    
        if num < 2:
            return "Junior"
        elif num <= 4:
            return "Semi-Senior"
        elif num < 8:
            return "Senior"
        else:
            return "Leader"
        
    else:
        return np.nan

In [9]:
df1["experience"] = df1["description"].apply(lambda x : find_years_of_experience(x))

df1["experience_levels"] = df1["experience"].apply(lambda x : experience_level(x))

# Considera los niveles (Junior <2, semisenior 2 - 4, senior 4-8, Leader 8<)

In [10]:
df1["experience"].isna().sum()

34872

In [11]:
strings = df1["description"].apply(lambda x : find_years_of_experience(x)).value_counts()


strings.index

Float64Index([2.0, 3.0, 5.0, 4.0, 6.0, 10.0, 7.0, 8.0, 1.0, 12.0, 11.0, 9.0], dtype='float64')

In [19]:
df3 = df1[["airtable_id", "experience", "experience_levels"]]

df3["experience"] = df3["experience"].astype(str)

C:\Users\danie\AppData\Local\Temp\ipykernel_13964\823670189.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["experience"] = df3["experience"].astype(str)


In [26]:
lista_puestos[lista_puestos["jobs"].str.contains("Rea")]

,jobs
382,Diseñador de interiores en Realidad Virtual
458,Desarrollador React


In [31]:
clean.skills[clean.skills.str.contains("Re")]

185                 NewRelic
228             React Native
229         React Router DOM
230    React Testing Library
231                  ReactJS
232                   Realms
233                    Redis
234                    Redux
235                Regresión
236      Rendimiento o carga
237              Replicación
238                  Requery
241                    Revel
Name: Competencia, dtype: object

In [43]:
new_skills = pd.concat([clean.skills, pd.Series(["React"])], ignore_index = True).apply(lambda x : x.lower())

In [44]:
df1["skills2"] = df1["description"].apply(lambda x : clean.get_skills(x.lower(), new_skills))

In [50]:
df1["skills2"].value_counts()

[]                                                                 23735
[sql]                                                               1249
[excel]                                                             1093
[linux]                                                              680
[integración]                                                        664
                                                                   ...  
[aws, bitbucket, github, integración, kubernetes, laravel, php]        1
[aws, git, java, jenkins, node, python, serverless]                    1
[cypress, git, javascript, scrum, react]                               1
[foundation, microservicios, python, rest, sql]                        1
[azure, git, java, scrum]                                              1
Name: skills2, Length: 4870, dtype: int64

In [51]:
df3 = df1[["airtable_id", "skills2"]]

df3

,airtable_id,skills2
0,rec007ki0rty4r45D,[]
1,rec008d5T9DjWTVBw,[php]
2,rec009ePhEZgwdUZF,"[django, javascript, microservicios, mongodb, ..."
3,rec00F8BVyguzTbVL,"[excel, html]"
4,rec00IDNdY6vDxqDb,"[bootstrap, git, html, javascript, jquery, php..."
...,...,...
49995,reczzkEh5UM4jSMYJ,[]
49996,reczzmUDDTai3yyxs,[]
49997,reczzn8dxb0KvZCcb,"[html, java]"
49998,reczznz9goqSvw6yL,[]


In [54]:
df3["skills2"].isna().sum()

0

In [58]:
%%time

for i in range(df3.shape[0]):
    
    records = {"records" : []}
    
    row = df3.iloc[i].to_dict()
    
    id_ = row["airtable_id"]
    
    row.pop("airtable_id")
    
    records["records"].append({"id"     : id_,
                               "fields" : {k : v for k, v in row.items()}})
    
    records["typecast"] = True

    # LATAM 001
    response = requests.patch(url = hab1.endpoint, json = records, headers = hab1.headers)

    print(f"response: {response.status_code}")
    
#     sleep(0.15)

    # LATAM 002
#     response = requests.patch(url = hab2.endpoint, json = records, headers = hab2.headers)

#     print(f"response: {response.status_code}")

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

...

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
response: 200
respon

In [55]:
response.json()

{'records': [{'id': 'rec008d5T9DjWTVBw',
   'createdTime': '2023-06-26T23:09:41.000Z',
   'fields': {'job_id': 'eyJqb2JfdGl0bGUiOiJUcmFiYWpvIERlc2RlIENhc2EgRGVzYXJyb2xsYWRvciBBbmRyb2lkIFJlZiAwMDQ2UyIsImh0aWRvY2lkIjoic243ZEpvNkdGeDBBQUFBQUFBQUFBQT09IiwidXVsZSI6IncrQ0FJUUlDSUxSV3dnVTJGc2RtRmtiM0kiLCJmYyI6IkVxSUNDdUlCUVVWek4ycE9VM1pFVFcxS2RETmlhR2R2VGtOa1JUSmFhMlpwWkROVFIybEZhMUZ0U21GSVdVOU5SV3BpVVVRM2RVUnJWSGx2TlhwMWRsZFdPSGh4Y0VNeGJrdElPRnBGVlV0blkwZFhjSGxFVlhSS1FUUmpZVzFCZFdsVVIyeFZZMk51ZW5sQ1ptRlRaREYwU1c0dFNrYzJka2RmUjBaRVNrRk5jbXRGVFRSR2NFWTVVMUpFVmxJNU1UZHVRblZuWjBrelkzRkdaR1l6V0ROd01XNXZlWE0zZDJKS2J6RjRTRVpuWlhoNWRWTTBkVUZYT0docVluY3pjMnBNWmpVMGREZEdXVk5IZGpCeGVYQXpTVTR4VjJoMmJVVlFlRU5DZWxWcWNtNHhhREZyUVJJWFNIcFhWVnBQUkZSSGN6WXRhMUJKVUhkdmRYZzBRVkVhSWtGUExUQnliRFJYWmtGV1RWWnFNVU5TU0ZGaGVsTnhORWM1VW5CamRIbGliMUUiLCJmY3YiOiIzIiwiZmNfaWQiOiJmY18xMiIsImFwcGx5X2xpbmsiOnsidGl0bGUiOiJBcHBseSBvbiBUZWNvbG9jby5jb20iLCJsaW5rIjoiaHR0cHM6Ly93d3cudGVjb2xvY28uY29tLzc1NDcxNC9wcm9ncmFtYWRvci5hc3B4

In [1]:
records

NameError: name 'records' is not defined

In [15]:
records = {"records" : []}
    
row = df3.iloc[i].to_dict()

id_ = row["airtable_id"]

row.pop("airtable_id")

records["records"].append({"id"     : id_,
                           "fields" : {k : None if pd.isna(v) else v for k, v in row.items()}})

records["typecast"] = True


In [16]:
from pprint import pprint
pprint(records, sort_dicts = False)

{'records': [{'id': 'rec0N0DEojoORBmjx',
              'fields': {'experience': 4.0,
                         'experience_levels': 'Semi-Senior'}}],
 'typecast': True}


In [17]:
################################################################################################################################